# DATA READING
Parquet file from bronze layer in Azure Datalake Gen2

In [0]:
df = spark.read.format('parquet')\
            .option('inferSchema', True)\
            .load('abfss://bronze@carsaleslake.dfs.core.windows.net/rawdata')

In [0]:
df.display()

# DATA TRANSFORMATION

Importing required libraries

In [0]:
from pyspark.sql.functions import *

Adding new columns

In [0]:
df = df.withColumn('model_catagory',split(col('Model_ID'),'-')[0])


In [0]:
df=df.withColumn('RevPerUnit',col('Revenue')/col('Units_Sold'))
df.display()


Percentage of Total Sales by Year

In [0]:
display(df.groupby('Year','BranchName').agg(sum('Units_Sold').alias('Total_units')).sort('Year','Total_units',ascending=[1,0]))

# DATA WRITING


Saves the transformed data to Silver layer in the Azure Data Lake Storage Gen2

In [0]:
df.write.format('parquet').mode('append').option('path','abfss://silver@carsaleslake.dfs.core.windows.net/carsales').save()

# Querying Silver Data

In [0]:
%sql
SELECT * FROM PARQUET.`abfss://silver@carsaleslake.dfs.core.windows.net/carsales`